# Pytorch图像分类模型转ONNX-水果30类

同济子豪兄 https://space.bilibili.com/1900783

代码运行云GPU平台：公众号 人工智能小技巧 回复 gpu

2022-8-22 2023-5-8

## 导入工具包

In [1]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

D:\Software\anaconda\envs\Pytorch\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


device cuda:0


## 下载自己训练得到的30类水果图像分类Pytorch模型权重文件

In [2]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth -P checkpoint


--2023-05-08 09:59:48--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_pytorch_20220814.pth
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 44854477 (43M) [binary/octet-stream]
正在保存至: “checkpoint/fruit30_pytorch_20220814.pth”

fruit30_pytorch_202 100%[===================>]  42.78M  23.1MB/s    用时 1.9s    

2023-05-08 09:59:51 (23.1 MB/s) - 已保存 “checkpoint/fruit30_pytorch_20220814.pth” [44854477/44854477])


## 导入训练好的模型

In [2]:
model = torch.load(r'E:\MV-Code-202018010103-Lucy\main\Train_Custom_Dataset\图像分类\3-【Pytorch】迁移学习训练自己的图像分类模型\checkpoint\best-1.000.pth')
model = model.eval().to(device)

## 构造一个输入图像Tensor

In [3]:
x = torch.randn(1, 3, 256, 256).to(device)

## 输入Pytorch模型推理预测，获得1000个类别的预测结果

In [4]:
output = model(x)

In [5]:
output.shape

torch.Size([1, 2])

## Pytorch模型转ONNX模型

In [6]:
x = torch.randn(1, 3, 256, 256).to(device)

with torch.no_grad():
    torch.onnx.export(
        model,                   # 要转换的模型
        x,                       # 模型的任意一组输入
        'resnet18_fruit30.onnx', # 导出的 ONNX 文件名
        opset_version=11,        # ONNX 算子集版本
        input_names=['input'],   # 输入 Tensor 的名称（自己起名字）
        output_names=['output']  # 输出 Tensor 的名称（自己起名字）
    ) 

## 验证onnx模型导出成功

In [7]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('resnet18_fruit30.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [8]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 2x912]
  %fc.bias[FLOAT, 2]
  %onnx::Conv_552[FLOAT, 32x3x3x3]
  %onnx::Conv_553[FLOAT, 32]
  %onnx::Conv_555[FLOAT, 72x32x1x1]
  %onnx::Conv_556[FLOAT, 72]
  %onnx::Conv_558[FLOAT, 72x32x1x1]
  %onnx::Conv_559[FLOAT, 72]
  %onnx::Conv_561[FLOAT, 72x24x3x3]
  %onnx::Conv_562[FLOAT, 72]
  %onnx::Conv_564[FLOAT, 72x72x1x1]
  %onnx::Conv_565[FLOAT, 72]
  %onnx::Conv_567[FLOAT, 72x72x1x1]
  %onnx::Conv_568[FLOAT, 72]
  %onnx::Conv_570[FLOAT, 72x24x3x3]
  %onnx::Conv_571[FLOAT, 72]
  %onnx::Conv_573[FLOAT, 72x72x1x1]
  %onnx::Conv_574[FLOAT, 72]
  %onnx::Conv_576[FLOAT, 168x72x1x1]
  %onnx::Conv_577[FLOAT, 168]
  %onnx::Conv_579[FLOAT, 168x72x1x1]
  %onnx::Conv_580[FLOAT, 168]
  %onnx::Conv_582[FLOAT, 168x24x3x3]
  %onnx::Conv_583[FLOAT, 168]
  %onnx::Conv_585[FLOAT, 168x168x1x1]
  %onnx::Conv_586[FLOAT, 168]
  %onnx::Conv_588[FLOAT, 168x168x1x1]
  %onnx::Conv_589[FLOAT, 168]
  %onnx::Conv_591[FLOAT, 168x24

## 使用Netron可视化模型结构

Netron：https://netron.app

视频教程：https://www.bilibili.com/video/BV1TV4y1P7AP